In all versions 3.x the narrative gets cleaned as before and products are cleaned once times by randomly removing etries with the three most common values for product. This run uses 5 different principle components. 

__Initialing the data__

In [1]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_first_cut.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 6


__Initializing the bag of words__

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [3]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=5)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(292470, 5)


In [4]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.44366181 -3.22165372  0.01535673  0.40540478  0.05836881]
 [ 3.81940412 -1.11041252  0.33335724 -1.00101786 -1.50928527]
 [11.86264377  4.43820391  4.55460654  0.51136761  3.20803623]
 ...
 [ 7.78326954  3.84639072 -3.26322614 -1.84297379  1.61026281]
 [ 1.0612983   0.45508499 -0.09378451 -0.35162597  0.21978829]
 [ 0.93802168  0.23182946  0.38587514 -0.34651002  0.13066908]]
                0         1         2         3         4
0        4.443662 -3.221654  0.015357  0.405405  0.058369
1        3.819404 -1.110413  0.333357 -1.001018 -1.509285
2       11.862644  4.438204  4.554607  0.511368  3.208036
3        2.077610  1.348508  1.202181 -0.664402 -2.075571
4        5.176561 -4.094794 -1.813807 -4.858488  1.603869
...           ...       ...       ...       ...       ...
292465   0.173739  0.226186 -0.251176  0.049035  0.072173
292466   5.128417 -4.206572 -0.406695  2.479155 -0.050453
292467   7.783270  3.846391 -3.263226 -1.842974  1.610263
292468   1.061298  0.455085 -0.09378

__One hot encoding of extra columns__

In [5]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(292470, 5)
(292470, 161)
(292470, 166)


__Split into train and test__

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(219352, 166)
(73118, 166)
(219352,)
(73118,)


__Application of linear SVM__

In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3717
                                                 Checking or savings account       0.99      1.00      1.00      4700
                                                               Consumer Loan       0.96      0.75      0.84      2965
                                                                 Credit card       1.00      1.00      1.00      4715
                                                 Credit card or prepaid card       0.96      0.97      0.96      7727
                                                            Credit reporting       1.00      1.00      1.00      7853
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.91      0.95     11816
                                                       